In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
import os
from data_utils import getDataSet, deprocessImage
import math
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.initializers import RandomNormal, Ones
from keras.optimizers import SGD
from keras.utils import to_categorical

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, classes_dic, mean = getDataSet('101_ObjectCategories/', 500, 1000)

In [ ]:
num_classes = len(classes_dic)
print ('X_train shape: {}'.format(X_train.shape))
print ('y_train shape: {}'.format(y_train.shape))
print ('X_val shape: {}'.format(X_val.shape))
print ('y_val shape: {}'.format(y_val.shape))
print ('X_test shape: {}'.format(X_test.shape))
print ('y_test shape: {}'.format(y_test.shape))
print ('number of classes: ' + str(num_classes))

In [ ]:
i = np.random.randint(0, 7145)
plt.imshow(np.uint8(deprocessImage(X_train[i], mean)))
print (classes_dic[y_train[i]])

In [ ]:
def AlexNetModel(num_classes):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    
    model.add(Conv2D(filters=256, kernel_size=(5, 5), kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    
    model.add(Conv2D(filters=384, kernel_size=(3, 3), kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), activation="relu", data_format="channels_last"))
    model.add(Conv2D(filters=384, kernel_size=(3, 3), kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), activation="relu", data_format="channels_last"))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), activation="relu", data_format="channels_last"))
    
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(units=4096, activation="relu", kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones()))
    model.add(Dropout(0.5))
    model.add(Dense(units=4096, activation="relu", kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones()))
    model.add(Dropout(0.5))
    
    model.add(Dense(units=num_classes, kernel_initializer=RandomNormal(stddev=0.01), bias_initializer=Ones(), activation="softmax"))
    
    return model

In [ ]:
model = AlexNetModel(num_classes)
optimizer = SGD(lr=0.007, momentum=0.9, decay=0.005, nesterov=True)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train, to_categorical(y_train, num_classes), epochs=30, batch_size=128, validation_data=(X_val, to_categorical(y_val, num_classes)))

In [ ]:
history = model.fit(X_train, to_categorical(y_train, num_classes), epochs=1, batch_size=128, validation_data=(X_val, to_categorical(y_val, num_classes)))
print (history.history)

In [ ]:
max_val_acc = max_val_acc
val_accs = val_accs
train_accs = train_accs
num_iters = num_iters
for i in range(50):
    num_iters += 1
    history = model.fit(X_train, to_categorical(y_train, num_classes), epochs=1, batch_size=128, validation_data=(X_val, to_categorical(y_val, num_classes)))
    if history.history['val_acc'][0] > max_val_acc:
        max_val_acc = history.history['val_acc'][0]
        model.save('model_keras.h5')
    val_accs.append(history.history['val_acc'][0])
    train_accs.append(history.history['acc'][0])

In [ ]:
print(max_val_acc)
print(num_iters)

In [ ]:
model.evaluate(X_val, to_categorical(y_val, num_classes))

In [ ]:
model.evaluate(X_test, to_categorical(y_test, num_classes))

In [ ]:
model.save('model_keras.h5')